In [1]:
# Imports.
import pandas as pd
import csv
import re
from tqdm import tqdm

In [4]:
# Get datasets.
grams = pd.read_csv("grams.csv")
print("Length of USDA portions dataset:", len(grams))
print("")

ingr = pd.read_csv("ingr.csv")
print("Lenght of recipes dataset:", len(ingr))
print("")

link = pd.read_csv("tlink.csv")
print("Lenght of link dataset:", len(link))
print("")

Length of USDA portions dataset: 32614

Lenght of recipes dataset: 30925

Lenght of link dataset: 16586



In [5]:
# This list will include grammage of recipes' ingredients and will be appended to ingr dataframe in the end.
grammage = []

In [6]:
# These lists represent each column in link dataframe. They will be used for matching food in ingr database with food in USDA database.
# Selective filtering will be done from cosine similarity scores as well.
# The lists are zipped to a list representing IngredientMatchScore abbreviated as ims.
li = []
lm = []
ls = []

for i in link.Ingr:
    li.append(i)
    
for i in link.Match:
    lm.append(i)
    
for i in link.Score:
    ls.append(i)

ims = list(zip(li, lm, ls))

In [7]:
# The initial for loop acts as an index for rows in ingr dataset. Iterates through rows.
for indx in tqdm(range(len(ingr))):
    
    # t represents the index number of the recipe on ingr dataframe. f, q, and u represent food, quantity, and unit columns on ingr dataframe respectively. gg represents grams as they are calculated; gn represents the ingredient name of the calculated grammage.
    # gn and gg will be zipped together then appended to grammage list on each iteration.
    t = []
    f = []
    q = []
    u = []
    gg = []
    gn = []
   
    # The following for loop iterates through unit list on the chosen row's unit column cell. Iterates through lists on unit column's cells for the row that the initial iteration is on. The elements are made singular and then appended to list u.
    for element in ingr.unit[indx].split(","):
        element = element.lower()
        c = re.sub(r"[\[\]]", "", element)
        c = c.replace("'", "")
        c = c.lstrip()
        if len(c) > 1:
            if c[-1] == "s":
                c = c[:-1]
        u.append(c)
        
    # The following for loop iterates through food list on the chosen row's food column cell. Iterates through lists on food column's cells for the row that the initial iteration is on. The elements are formatted to match their appearances on the link dataframe and then appended to list f.
    for element in ingr.food[indx].split(","):
        element = element.lower()
        c = re.sub(r"[\[\]]", "", element)
        c = c.replace("'", "")
        c = c.lstrip()
        f.append(c)
        
    # The following for loop iterates through quantity list on the chosen row's quantity column cell. Iterates through lists on quantity column's cells for the row that the initial iteration is on. The elements are formatted to appear as one integer item and then appended to list q.
    # This loop also appends the index number (indx) to list t.
    for element in ingr.quantity[indx].split(","):
        element = element.lower()
        c = re.sub(r"[\[\]]", "", element)
        c = c.replace("'", "")
        c = c.lstrip()
        c = re.findall(r'^\D*(\d+)', c)
        q.append(c)
        t.append(indx)
        
    # The above iterations' lists are zipped together to form TitleFoodQuantityUnit list, abbreviated as tfqu.
    tfqu = list(zip(t,f,q,u))
    
    # The following loop iterates through list tfqu. Each iteration has one food item, its quantity and its unit.
    for i in tfqu:
        
        # Type of each i is tuple which is not callable. Thus, i's are converted to list type variables.
        i = list(i)
        
        # This nested if statement converts pound unit type to oz unit type since the USDA dataset does not have pound unit type in its portion descriptions.
        if i[3] == "pound" or i[3] == "pounds" or i[3] == "lb." or i[3] == "lbs." or i[3] == "lb" or i[3] == "lbs":
            i[3] = "oz"
            if type(i[2]) == list:
                if len(i[2]) == 0:
                    continue
                else:
                    i[2][0] = float(i[2][0])*16
            else:
                i[2] = float(i[2])*16
        
        # All items are initially matched to "NS", an abbreviation for "not specified". This handles a ValueError in the following loop.
        matched = "NS"
        
        # The following loop iterates through matches present in list ims.
        for element in ims:
            
            # The following nested if statement manually matches possible mismatched food items that have the potential to be main ingredients. Matches under the score of 0.5 were observed to be mostly mismatched in the categories given below and thus they are manually matched to sensible items on USDA dataset.
            # All the other food items are matched to their respective counterparts on the link dataset.
            if i[1] == element[0]:
                score = element[2]
                if "chicken" in i[1] and score <= 0.5:
                    matched = "chicken breast, baked, broiled, or roasted, skin eaten, from raw"
                elif "fish" in i[1] and score <= 0.5:
                    matched = "fish, ns as to type, cooked, ns as to cooking method"
                elif "steak" in i[1] and score <= 0.5:
                    matched = "steak, ns as to type of meat, cooked, ns as to fat eaten"
                elif "beef" in i[1] and score <= 0.5:
                    matched = "beef, ns as to cut, cooked, ns as to fat eaten"
                elif "egg" in i[1]:
                    if "noodle" in i[1]:
                        matched = "beef and egg noodles, baby food, ns as to strained or junior"
                    elif "plant" in i[1]:
                        matched = "eggplant, raw"
                    elif score <= 0.5:
                        matched = "egg, whole, raw"
                elif "flour" in i[1] and score <= 0.5:
                    matched = "bread, white"
                else:
                    matched = element[1]
                
                # USDA dataset is filtered for the provided matched food item.
                df = grams.loc[grams["Main_Food_Description"] == matched]
                
                # The following loop iterates through the filtered dataframe's portion description column in order to match unit types. If units are not matched, the food item was likely to be mismatched in the first place and thus discarded.
                for row in df.Portion_Description:
                    
                    # The following nested if statement further filters the filtered dataframe (df) to unit type matches.
                    # Then grammage is calculated by multiplying portion weight value from the first row of the further filtered dataframe and quantity value from i in tfqu. The numerical value is appended to gg while the name of the item is appended to gn.
                    if i[3] in row:
                        inner_df = df.loc[df["Portion_Description"] == row]
                        
                        # Quantity values could appear as empty lists as well as regular lists, so that is handled below as well.
                        if type(i[2]) == list:
                            if len(i[2]) == 0:
                                gg.append(0)
                                gn.append(i[1])
                                break
                            else:
                                gg.append(float(i[2][0])*float(inner_df[inner_df.columns[5]].iloc[0]))
                                gn.append(i[1])
                                break
                        else:
                            gg.append(float(i[2])*float(inner_df[inner_df.columns[5]].iloc[0]))
                            gn.append(i[1])
                            break
                
    # gn and gg are zipped to master grams list abbreviated as gm.            
    gm = list(zip(gn, gg))         
    
    # Each iteration creates a different list in gm where they are then appended to grammage to get a list of lists for each recipe.
    grammage.append(gm)
    
    # Index is increased by one at the iteration's end.
    indx+=1

100%|██████████| 30925/30925 [17:36<00:00, 29.27it/s]


In [9]:
# grammage list is added to ingr dataframe.
ingr["grams"] = grammage

# New ingr dataframe is converted to a csv file.
ingr.to_csv("grammage.csv", encoding='utf-8', index=False)